In [2]:
import h5py
import numpy as np
from PIL import Image
import json
import os.path as osp

In [3]:
from dataset import SceneTextDataset
import albumentations as A
from torch.utils.data import Dataset

import os
import os.path as osp
import json
import numpy as np
from shapely.geometry import Polygon
from dataset import resize_img, adjust_height, rotate_img, crop_img, generate_roi_mask
import albumentations as A
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.20 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
INFO:matplotlib.font_manager:generated new fontManager


In [5]:
_lang_list = ['chinese', 'japanese', 'thai', 'vietnamese']
root_dir = os.environ.get('SM_CHANNEL_TRAIN', 'data')
split = 'train'
total_anno = dict(images=dict())
for nation in _lang_list:
    with open(osp.join(root_dir, '{}_receipt/ufo/{}.json'.format(nation, split)), 'r', encoding='utf-8') as f:
        anno = json.load(f)
    for im in anno['images']:
        total_anno['images'][im] = anno['images'][im]

anno = total_anno
image_fnames = sorted(anno['images'].keys())


In [6]:
def _infer_dir(fname):
    lang_indicator = fname.split('.')[1]
    if lang_indicator == 'zh':
        lang = 'chinese'
    elif lang_indicator == 'ja':
        lang = 'japanese'
    elif lang_indicator == 'th':
        lang = 'thai'
    elif lang_indicator == 'vi':
        lang = 'vietnamese'
    else:
        raise ValueError
    return osp.join(root_dir, f'{lang}_receipt', 'img', split)


In [7]:
ignore_under_threshold=10
drop_under_threshold=1
image_size=2048
crop_size=1024

In [8]:
train_dataset = SceneTextDataset(
root_dir,
split='train',
image_size=image_size,
crop_size=crop_size,)

In [11]:
from east_dataset import generate_score_geo_maps
import cv2
import torch

In [36]:
%%time 
map_scale=0.5
# for idx in range(1,2):
idx = 3
image, word_bboxes, roi_mask = train_dataset[idx]
score_map, geo_map = generate_score_geo_maps(image, word_bboxes, map_scale=map_scale)

mask_size = int(image.shape[0] * map_scale), int(image.shape[1] * map_scale)
roi_mask = cv2.resize(roi_mask, dsize=mask_size)
if roi_mask.ndim == 2:
    roi_mask = np.expand_dims(roi_mask, axis=2)

if True:
    image = torch.Tensor(image).permute(2, 0, 1)
    score_map = torch.Tensor(score_map).permute(2, 0, 1)
    geo_map = torch.Tensor(geo_map).permute(2, 0, 1)
    roi_mask = torch.Tensor(roi_mask).permute(2, 0, 1)


CPU times: user 4.57 s, sys: 4.61 s, total: 9.18 s
Wall time: 1.41 s
